# Introduction

In the project, I want to use web scraping to get the data from a website, in order to collaborate the information about which state/county announced stay at home policy and when the policy started

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 1000)
import numpy as np
import pyodbc
import datetime
import copy
import matplotlib.pyplot as plt
import time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import urllib3
from bs4 import BeautifulSoup

Download the data from https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html website

In [3]:
import urllib.request

with urllib.request.urlopen("https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html") as url:
    s = url.read()
    print(s)

b'<!DOCTYPE html>\n<html lang="en" class="nytapp-vi-interactive" xmlns:og="http://opengraphprotocol.org/schema/">\n  <head>\n    <title data-rh="true">See Which States and Cities Have Told Residents to Stay at Home - The New York Times</title>\n    <meta data-rh="true" property="article:published" content="2020-03-24T02:05:48.000Z"/><meta data-rh="true" property="article:modified" content="2020-04-07T15:30:51.347Z"/><meta data-rh="true" http-equiv="Content-Language" content="en"/><meta data-rh="true" name="robots" content="noarchive"/><meta data-rh="true" name="articleid" content="100000007050012"/><meta data-rh="true" name="nyt_uri" content="nyt://interactive/730ca6de-35df-5501-9b2a-ce6e35bb8abe"/><meta data-rh="true" name="pubp_event_id" content="pubp://event/c5446806a6cd4d388e8c4f794b98132b"/><meta data-rh="true" name="description" content="In an attempt to stop the spread of the coronavirus, a vast majority of states and the Navajo Nation have given directives, affecting about nine

In [4]:
soup = BeautifulSoup(s, 'lxml')
soup

<!DOCTYPE html>
<html class="nytapp-vi-interactive" lang="en" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title data-rh="true">See Which States and Cities Have Told Residents to Stay at Home - The New York Times</title>
<meta content="2020-03-24T02:05:48.000Z" data-rh="true" property="article:published"/><meta content="2020-04-07T15:30:51.347Z" data-rh="true" property="article:modified"/><meta content="en" data-rh="true" http-equiv="Content-Language"/><meta content="noarchive" data-rh="true" name="robots"/><meta content="100000007050012" data-rh="true" name="articleid"/><meta content="nyt://interactive/730ca6de-35df-5501-9b2a-ce6e35bb8abe" data-rh="true" name="nyt_uri"/><meta content="pubp://event/c5446806a6cd4d388e8c4f794b98132b" data-rh="true" name="pubp_event_id"/><meta content="In an attempt to stop the spread of the coronavirus, a vast majority of states and the Navajo Nation have given directives, affecting about nine in 10 U.S. residents." data-rh="true" name="descr

In [5]:
state_county=[]
for link in soup.find_all(class_='state-wrap'):
    for i,q,a in zip(link.find_all('h3'), link.find_all(class_='l-place'), link.find_all(class_='l-date')):
        state_county.append(i.text)
        state_county.append(q.text)
        state_county.append(a.text)

In [6]:
state_county

['Oklahoma',
 'Claremore About 19,000 people',
 ', effective April 6 at 8 a.m.',
 'Utah',
 'Davis County About 352,000 people',
 ', effective April 1 at 11:59 p.m.',
 'Wyoming',
 'Jackson About 10,000 people',
 ', effective March 28']

In [7]:
county=[]
for link in soup.find_all(class_='state-wrap'):
    for q,a in zip(link.find_all(class_='l-place'), link.find_all(class_='l-date')):
        county.append(q.text)
        county.append(a.text)

In [8]:
county

['Claremore About 19,000 people',
 ', effective April 6 at 8 a.m.',
 'Edmond About 93,000 people',
 ', effective March 30 at 11:59 p.m.',
 'Moore About 62,000 people',
 ', effective April 4 at 11:59 p.m.',
 'Norman About 123,000 people',
 ', effective March 25 at 11:59 p.m.',
 'Oklahoma City About 649,000 people',
 ', effective March 28 at 11:59 p.m.',
 'Sallisaw About 9,000 people',
 ', effective April 4 at 11:59 p.m.',
 'Stillwater About 50,000 people',
 ', effective March 30 at 11:59 p.m.',
 'Tulsa About 401,000 people',
 ', effective March 28 at 11:59 p.m.',
 'Davis County About 352,000 people',
 ', effective April 1 at 11:59 p.m.',
 'Salt Lake County About 1.2 million people',
 ', effective March 30 at 12:01 a.m.',
 'Summit County About 42,000 people',
 ', effective March 27 at 12:01 a.m.',
 'Jackson About 10,000 people',
 ', effective March 28']

In [9]:
state = []
for link in soup.find_all(class_='state-wrap statewide'):
    for i,a in zip(link.find_all('h3'), link.find_all(class_='l-date')):
        state.append(i.text)
        state.append(a.text)
state

['Alabama About 4.9 million people',
 ', effective April 4 at 5 p.m.',
 'Alaska About 737,000 people',
 ', effective March 28 at 5 p.m.',
 'Arizona About 7.2 million people',
 ', effective March 31 at 5 p.m.',
 'California About 39.6 million people',
 ', effective March 19',
 'Colorado About 5.7 million people',
 ', effective March 26 at 6 a.m.',
 'Connecticut About 3.6 million people',
 ', effective March 23 at 8 p.m.',
 'Delaware About 973,000 people',
 ', effective March 24 at 8 a.m.',
 'District of Columbia About 702,000 people',
 ', effective April 1 at 12:01 a.m.',
 'Florida About 21.5 million people',
 ', effective April 3 at 12:01 a.m.',
 'Georgia About 10.6 million people',
 ', effective April 3',
 'Hawaii About 1.4 million people',
 ', effective March 25 at 12:01 a.m.',
 'Idaho About 1.8 million people',
 ', effective March 25 at 1:30 p.m.',
 'Illinois About 12.7 million people',
 ', effective March 21 at 5 p.m.',
 'Indiana About 6.7 million people',
 ', effective March 24 at

In [10]:
state[0]

'Alabama About 4.9 million people'

In [11]:
state1 = pd.DataFrame()
s = []
d = []
for i in range(len(state)):
    if (i % 2) == 0:
        st = state[i].split(' About')[0]
        s.append(st)
    else:
        dt = state[i].split('at')[0]
        dt1 = dt.replace(", effective","")
        dt2 = dt1.replace(" ", "")
        dt3 = dt2.replace("March", "2020-03-")
        dt4 = dt3.replace("April", "2020-04-")
        d.append(dt4)
state1['State'] = s
state1['Effecitive Date'] = d
state1

,State,Effecitive Date
0,Alabama,2020-04-4
1,Alaska,2020-03-28
2,Arizona,2020-03-31
3,California,2020-03-19
4,Colorado,2020-03-26
5,Connecticut,2020-03-23
6,Delaware,2020-03-24
7,District of Columbia,2020-04-1
8,Florida,2020-04-3
9,Georgia,2020-04-3


In [12]:
state_county1 = []
for i in state_county:
    if 'effective' in i:
        del i
    else: 
        c = i.split(' About')[0]
        state_county1.append(c)

state_county1

['Oklahoma', 'Claremore', 'Utah', 'Davis County', 'Wyoming', 'Jackson']

In [13]:
county1 = pd.DataFrame()
s = []
d = []
for i in range(len(county)):
    if (i % 2) == 0:
        st = county[i].split(' About')[0]
        s.append(st)
    else:
        dt = county[i].split('at')[0]
        dt1 = dt.replace(", effective","")
        dt2 = dt1.replace(" ", "")
        dt3 = dt2.replace("March", "2020-03-")
        dt4 = dt3.replace("April", "2020-04-")
        d.append(dt4)
county1['County'] = s
county1['Effecitive Date'] = d
county1

,County,Effecitive Date
0,Claremore,2020-04-6
1,Edmond,2020-03-30
2,Moore,2020-04-4
3,Norman,2020-03-25
4,Oklahoma City,2020-03-28
5,Sallisaw,2020-04-4
6,Stillwater,2020-03-30
7,Tulsa,2020-03-28
8,Davis County,2020-04-1
9,Salt Lake County,2020-03-30


In [14]:
county1.County[county1.County == 'Birmingham'].index.tolist()

[]

In [15]:
county1['State'] = 0
county1[1:2]

,County,Effecitive Date,State
1,Edmond,2020-03-30,0


In [16]:
county1.iloc[0,0]

'Claremore'

In [17]:
begin = county1.County[county1.County == county1.iloc[0,0]].index.tolist()[0]
end = county1.County[county1.County == county1.iloc[-1,0]].index.tolist()[0]

In [18]:
county1[begin:end]

,County,Effecitive Date,State
0,Claremore,2020-04-6,0
1,Edmond,2020-03-30,0
2,Moore,2020-04-4,0
3,Norman,2020-03-25,0
4,Oklahoma City,2020-03-28,0
5,Sallisaw,2020-04-4,0
6,Stillwater,2020-03-30,0
7,Tulsa,2020-03-28,0
8,Davis County,2020-04-1,0
9,Salt Lake County,2020-03-30,0


In [19]:
for i in range(len(state_county1)):
    for j in range(len(county1.County)):
        if state_county1[i] == county1.County[j] and (i+2) < len(state_county1):
            begin = county1.County[county1.County == state_county1[i]].index.tolist()[0]
            end = county1.County[county1.County == state_county1[i+2]].index.tolist()[0]
            print(end)
            county1['State'][begin:end] = state_county1[i-1]
        else:
            county1['State'][end:] = state_county1[i-1]
            
            

8
11


C:\Users\YuLiu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [20]:
county1['Effecitive Date'] = pd.to_datetime(county1['Effecitive Date'])
county2 = county1[['State', 'County', 'Effecitive Date']]
county2

,State,County,Effecitive Date
0,Oklahoma,Claremore,2020-04-06
1,Oklahoma,Edmond,2020-03-30
2,Oklahoma,Moore,2020-04-04
3,Oklahoma,Norman,2020-03-25
4,Oklahoma,Oklahoma City,2020-03-28
5,Oklahoma,Sallisaw,2020-04-04
6,Oklahoma,Stillwater,2020-03-30
7,Oklahoma,Tulsa,2020-03-28
8,Utah,Davis County,2020-04-01
9,Utah,Salt Lake County,2020-03-30


In [21]:
state1['Effecitive Date'] = pd.to_datetime(state1['Effecitive Date'])
state1['County'] = state1['State']
state2 = state1[['State', 'County', 'Effecitive Date']]
state2

,State,County,Effecitive Date
0,Alabama,Alabama,2020-04-04
1,Alaska,Alaska,2020-03-28
2,Arizona,Arizona,2020-03-31
3,California,California,2020-03-19
4,Colorado,Colorado,2020-03-26
5,Connecticut,Connecticut,2020-03-23
6,Delaware,Delaware,2020-03-24
7,District of Columbia,District of Columbia,2020-04-01
8,Florida,Florida,2020-04-03
9,Georgia,Georgia,2020-04-03


In [22]:
state3 = state2.append(county2)
state3

,State,County,Effecitive Date
0,Alabama,Alabama,2020-04-04
1,Alaska,Alaska,2020-03-28
2,Arizona,Arizona,2020-03-31
3,California,California,2020-03-19
4,Colorado,Colorado,2020-03-26
5,Connecticut,Connecticut,2020-03-23
6,Delaware,Delaware,2020-03-24
7,District of Columbia,District of Columbia,2020-04-01
8,Florida,Florida,2020-04-03
9,Georgia,Georgia,2020-04-03


In [23]:
abv = pd.DataFrame([{
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}]).transpose().reset_index()
abv.columns = ['Abbrev', 'State']
abv

,Abbrev,State
0,AK,Alaska
1,AL,Alabama
2,AR,Arkansas
3,AS,American Samoa
4,AZ,Arizona
5,CA,California
6,CO,Colorado
7,CT,Connecticut
8,DC,District of Columbia
9,DE,Delaware


In [24]:
abv.Abbrev = abv.Abbrev.str.strip()
state1['State'] = state1['State'].str.strip()

In [25]:
state3[state3['State'].isin(abv['State'])] = state3[state3['State'].isin(abv['State'])].assign(State=lambda x: x['State'].map(abv.set_index('State')['Abbrev']))
state3

,State,County,Effecitive Date
0,AL,Alabama,2020-04-04
1,AK,Alaska,2020-03-28
2,AZ,Arizona,2020-03-31
3,CA,California,2020-03-19
4,CO,Colorado,2020-03-26
5,CT,Connecticut,2020-03-23
6,DE,Delaware,2020-03-24
7,DC,District of Columbia,2020-04-01
8,FL,Florida,2020-04-03
9,GA,Georgia,2020-04-03


In [26]:
state3[state3['County'].isin(abv['State'])] = state3[state3['County'].isin(abv['State'])].assign(County=lambda x: x['County'].map(abv.set_index('State')['Abbrev']))
state3

,State,County,Effecitive Date
0,AL,AL,2020-04-04
1,AK,AK,2020-03-28
2,AZ,AZ,2020-03-31
3,CA,CA,2020-03-19
4,CO,CO,2020-03-26
5,CT,CT,2020-03-23
6,DE,DE,2020-03-24
7,DC,DC,2020-04-01
8,FL,FL,2020-04-03
9,GA,GA,2020-04-03
